# HW 6
# Fraud Analytics
# Songqiao Li

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl
import calendar
import timeit
from datetime import datetime
from datetime import timedelta
import datetime as dt

# 

# Clean Data

In [117]:
data = pd.read_csv('card transactions.csv')


In [118]:
data

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
...,...,...,...,...,...,...,...,...,...,...
96748,96749,5142276053,2006-12-31,3500000006160,BEST BUY 00001610,KY,41042.0,P,84.79,0
96749,96750,5142225701,2006-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248.0,P,118.75,0
96750,96751,5142226486,2006-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,363.56,0
96751,96752,5142244619,2006-12-31,8834000695412,BUY.COM,CA,92656.0,P,2202.03,0


# 

## Remove all but P

In [119]:
data = data[data['Transtype'] == "P"]
data

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
...,...,...,...,...,...,...,...,...,...,...
96748,96749,5142276053,2006-12-31,3500000006160,BEST BUY 00001610,KY,41042.0,P,84.79,0
96749,96750,5142225701,2006-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248.0,P,118.75,0
96750,96751,5142226486,2006-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,363.56,0
96751,96752,5142244619,2006-12-31,8834000695412,BUY.COM,CA,92656.0,P,2202.03,0


## Single largest transaction outlier

In [120]:
data.Amount.describe()

count    9.639800e+04
mean     4.280340e+02
std      1.002446e+04
min      1.000000e-02
25%      3.346000e+01
50%      1.379800e+02
75%      4.282500e+02
max      3.102046e+06
Name: Amount, dtype: float64

In [121]:
data.Amount.max()

3102045.53

In [122]:
data[data['Amount'] == data.Amount.max()]

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
52714,52715,5142189135,2006-07-13,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0


In [123]:
data = data.drop(labels=52714, axis=0)
data

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
...,...,...,...,...,...,...,...,...,...,...
96748,96749,5142276053,2006-12-31,3500000006160,BEST BUY 00001610,KY,41042.0,P,84.79,0
96749,96750,5142225701,2006-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248.0,P,118.75,0
96750,96751,5142226486,2006-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,363.56,0
96751,96752,5142244619,2006-12-31,8834000695412,BUY.COM,CA,92656.0,P,2202.03,0


# 

## Fill in missing fields

In [124]:
# Full in Merchnum
merchdes_merchnum = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_merchnum :
        merchdes_merchnum[merchdes] = data.loc[index, 'Merchnum']

# fill in by mapping with Merch description
data['Merchnum'] = data['Merchnum'].fillna(data['Merch description'].map(merchdes_merchnum))

# assign unknown for adjustments transactions
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merchnum'] = data['Merchnum'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

In [125]:
# Fill in Merch state
# dict for mapping
zip_state = {}
for index, zip5 in data[data['Merch zip'].notnull()]['Merch zip'].items():
    if zip5 not in zip_state:
        zip_state[zip5] = data.loc[index, 'Merch state']
        
zip_state['00926'] = 'PR'
zip_state['00929'] = 'PR'
zip_state['00934'] = 'PR'
zip_state['00902'] = 'PR'
zip_state['00738'] = 'PR'
zip_state['90805'] = 'CA'
zip_state['76302'] = 'TX'
zip_state['00914'] = 'PR'
zip_state['95461'] = 'CA'
zip_state['00680'] = 'PR'
zip_state['00623'] = 'PR'
zip_state['00726'] = 'PR'
zip_state['00936'] = 'PR'
zip_state['12108'] = 'NY'
zip_state['00791'] = 'PR'
zip_state['00907'] = 'PR'
zip_state['00922'] = 'PR'
zip_state['00920'] = 'PR'
zip_state['00801'] = 'VI'
zip_state['31040'] = 'GA'
zip_state['41160'] = 'KY'
zip_state['00681'] = 'PR'
merchnum_state = {}
for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_state :
        merchnum_state [merchnum] = data.loc[index, 'Merch state']
merchdes_state = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_state :
        merchdes_state [merchdes] = data.loc[index, 'Merch state']
                
# fill in by mapping with zip, merchnum and merch description
data['Merch state'] = data['Merch state'].fillna(data['Merch zip'].map(zip_state))
data['Merch state'] = data['Merch state'].fillna(data['Merchnum'].map(merchnum_state))
data['Merch state'] = data['Merch state'].fillna(data['Merch description'].map(merchdes_state))
                                             
# assign unknown for adjustments transactions
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch state'] = data['Merch state'].mask(data['Merch description'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')

In [126]:
# Fill in Merch zip
merchnum_zip = {}
for index, merchnum in data[data['Merchnum'].notnull()]['Merchnum'].items():
    if merchnum not in merchnum_zip :
        merchnum_zip [merchnum] = data.loc[index, 'Merch zip']
merchdes_zip = {}
for index, merchdes in data[data['Merch description'].notnull()]['Merch description'].items():
    if merchdes not in merchdes_zip :
        merchdes_zip [merchdes] = data.loc[index, 'Merch zip']
                                           
# fill in by mapping with merchnum and merch description
data['Merch zip'] = data['Merch zip'].fillna(data['Merchnum'].map(merchnum_zip))
data['Merch zip'] = data['Merch zip'].fillna(data['Merch description'].map(merchdes_zip))
                                           
# assign unknown for adjustments transactions
data['Merch zip'] = data['Merch zip'].mask(data['Merch zip'] == 'RETAIL CREDIT ADJUSTMENT', 'unknown')
data['Merch zip'] = data['Merch zip'].mask(data['Merch zip'] == 'RETAIL DEBIT ADJUSTMENT', 'unknown')
                                           
# fill the rest with unknown
data = data.fillna('unknown')
data.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Recnum             96397 non-null  int64  
 1   Cardnum            96397 non-null  int64  
 2   Date               96397 non-null  object 
 3   Merchnum           96397 non-null  object 
 4   Merch description  96397 non-null  object 
 5   Merch state        96397 non-null  object 
 6   Merch zip          96397 non-null  object 
 7   Transtype          96397 non-null  object 
 8   Amount             96397 non-null  float64
 9   Fraud              96397 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 10.1+ MB


In [127]:
data

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
...,...,...,...,...,...,...,...,...,...,...
96748,96749,5142276053,2006-12-31,3500000006160,BEST BUY 00001610,KY,41042.0,P,84.79,0
96749,96750,5142225701,2006-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248.0,P,118.75,0
96750,96751,5142226486,2006-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,363.56,0
96751,96752,5142244619,2006-12-31,8834000695412,BUY.COM,CA,92656.0,P,2202.03,0


# 

# Create Varaibles

## DOW

In [128]:
# Find the day of the week
data['Dow'] = data['Date'].apply(lambda x: calendar.day_name[datetime.strptime(x, '%Y-%m-%d').weekday()])

In [129]:
# We want to not use the oot for target encoding variables
train_test = data[data['Date'] < '2010-09-01']
c = 4; nmid = 20; y_avg = train_test['Fraud'].mean()
y_dow = train_test.groupby('Dow')['Fraud'].mean()
num = train_test.groupby('Dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/c))
data['Dow_Risk'] = data.Dow.map(y_dow_smooth)

In [130]:
y_dow_smooth.reset_index(name = 'Risk')

,Dow,Risk
0,Friday,0.009530
1,Monday,0.010810
2,Saturday,0.018797
3,Sunday,0.022419
4,Thursday,0.007964
5,Tuesday,0.010444
6,Wednesday,0.008749


In [131]:
# trans date type
data['Date'] = pd.to_datetime(data['Date'])

In [132]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Recnum             96397 non-null  int64         
 1   Cardnum            96397 non-null  int64         
 2   Date               96397 non-null  datetime64[ns]
 3   Merchnum           96397 non-null  object        
 4   Merch description  96397 non-null  object        
 5   Merch state        96397 non-null  object        
 6   Merch zip          96397 non-null  object        
 7   Transtype          96397 non-null  object        
 8   Amount             96397 non-null  float64       
 9   Fraud              96397 non-null  int64         
 10  Dow                96397 non-null  object        
 11  Dow_Risk           96397 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 11.6+ MB


# 

## Pasting Vars

In [133]:
data['Cardnum_Merchnum'] = data['Cardnum'].astype(str) + data['Merchnum'].astype(str)
data['Cardnum_zip'] = data['Cardnum'].astype(str) + data['Merch zip'].astype(str)
data['Cardnum_state'] = data['Cardnum'].astype(str) + data['Merch state'].astype(str)

In [134]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96397 entries, 0 to 96752
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Recnum             96397 non-null  int64         
 1   Cardnum            96397 non-null  int64         
 2   Date               96397 non-null  datetime64[ns]
 3   Merchnum           96397 non-null  object        
 4   Merch description  96397 non-null  object        
 5   Merch state        96397 non-null  object        
 6   Merch zip          96397 non-null  object        
 7   Transtype          96397 non-null  object        
 8   Amount             96397 non-null  float64       
 9   Fraud              96397 non-null  int64         
 10  Dow                96397 non-null  object        
 11  Dow_Risk           96397 non-null  float64       
 12  Cardnum_Merchnum   96397 non-null  object        
 13  Cardnum_zip        96397 non-null  object        
 14  Cardnu

In [135]:
data

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,Dow,Dow_Risk,Cardnum_Merchnum,Cardnum_zip,Cardnum_state
0,1,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,Sunday,0.022419,51421904395509006296254,514219043938118.0,5142190439TN
1,2,5142183973,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,Sunday,0.022419,514218397361003026333,51421839731803.0,5142183973MA
2,3,5142131721,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,Sunday,0.022419,51421317214503082993600,514213172120706.0,5142131721MD
3,4,5142148452,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,Sunday,0.022419,51421484525509006296254,514214845238118.0,5142148452TN
4,5,5142190439,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,Sunday,0.022419,51421904395509006296254,514219043938118.0,5142190439TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96748,96749,5142276053,2006-12-31,3500000006160,BEST BUY 00001610,KY,41042.0,P,84.79,0,Sunday,0.022419,51422760533500000006160,514227605341042.0,5142276053KY
96749,96750,5142225701,2006-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248.0,P,118.75,0,Sunday,0.022419,51422257018090710030950,514222570145248.0,5142225701OH
96750,96751,5142226486,2006-12-31,4503057341100,"TECH PAC, INC",OH,45150.0,P,363.56,0,Sunday,0.022419,51422264864503057341100,514222648645150.0,5142226486OH
96751,96752,5142244619,2006-12-31,8834000695412,BUY.COM,CA,92656.0,P,2202.03,0,Sunday,0.022419,51422446198834000695412,514224461992656.0,5142244619CA


### first three zip

In [136]:
data['zip3'] = [str(i)[:3] for i in data['Merch zip']]

In [137]:
zip_first_30 = data[data['Fraud'] == 1]['zip3'].value_counts()[:30]#.index
is_30 = []
for i in data['zip3']:
    if i not in zip_first_30.index:
        is_30 += ["0"]
    else:
        is_30 += [i]

data['isin 30'] = is_30

### avg risk amount

In [138]:
data['avg_risk_amt'] = data['Amount'] * data['Dow_Risk']

### Month & Year

In [139]:
data['Month'] = [i.month for i in data['Date']]

In [140]:
data['Year'] = [i.year for i in data['Date']]

### Merch desc

In [141]:
data['abbre_merch_desc'] = [i[:5] for i in data['Merch description']]
data[data['Fraud'] == 1]['abbre_merch_desc'].value_counts()

AMAZO    59
FEDEX    51
ACI*A    48
STEVE    47
DIRKS    45
         ..
ANALY     1
CC *P     1
CABLE     1
MITY      1
BIG 5     1
Name: abbre_merch_desc, Length: 230, dtype: int64

In [142]:
popular_100_desc = pd.DataFrame(data['abbre_merch_desc'].value_counts()).head(100)
popular_100_desc

,abbre_merch_desc
FEDEX,11775
OFFIC,2744
STAPL,2656
FISHE,2402
GSA/C,1727
...,...
RAINI,132
BLACK,131
LOWE',130
PHOTO,129


In [143]:
popular_100_desc.index

Index(['FEDEX', 'OFFIC', 'STAPL', 'FISHE', 'GSA/C', 'GSA-F', 'SIGMA', 'MWI*M',
       'FRANK', 'CDW*G', 'DELL ', 'AMAZO', 'VWR S', 'RETAI', 'PC *P', 'COMPU',
       'A DAI', 'C & C', 'GLOBA', 'BUY.C', 'PROFE', 'AMERI', 'NATIO', 'LAB S',
       'GOVER', 'SOCIE', 'COLE ', 'MILLE', 'LABSO', 'GRAIN', 'THE L', 'HOME ',
       'MC MA', 'GTSI', 'AGILE', 'KINKO', 'WRIGH', 'BARNE', 'IBM I', 'XEROX',
       'ENVIR', 'BEST ', 'IMPAC', 'RADIO', 'MAG S', 'MICRO', 'CANON', 'QUILL',
       'LIFE ', 'TECH ', 'VIKIN', 'CC *P', 'NEWAR', 'FREDP', 'WEST ', 'AMERS',
       'INTER', 'BIO-R', 'HOLID', 'ACI*A', 'CORP ', 'STAY ', 'PREST', 'AMC S',
       'RITZ ', 'IWP N', 'STAR ', 'NATL ', 'WAL M', 'SKILL', 'GRADU', 'APPLI',
       'UNICO', 'NORTO', 'RESEA', 'HAWOR', 'WATER', 'KRACK', 'USGPO', 'RESTE',
       'B.A. ', 'PRICE', 'ADVAN', 'ESRI', 'SCOTT', 'THOMA', 'UNIV ', 'BT OF',
       'AMER ', 'OMNIF', 'DAN K', 'USO*F', 'NORTH', 'BOISE', 'DIONE', 'RAINI',
       'BLACK', 'LOWE'', 'PHOTO', 'BMC*B'],
      dtyp

In [144]:
is_100 = []
for i in data['abbre_merch_desc']:
    if i not in popular_100_desc.index:
        is_100 += [0]
    else:
        is_100 += [i]

data['isin 100'] = is_100

### Types

In [145]:
data['avg_risk_amt'] = data['avg_risk_amt'].astype('object')
data['Month'] = data['Month'].astype('object')
data['Year'] = data['Year'].astype('object')
data['isin 100'] = data['isin 100'].astype('object')

In [146]:
data.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                    object
Transtype                    object
Amount                      float64
Fraud                         int64
Dow                          object
Dow_Risk                    float64
Cardnum_Merchnum             object
Cardnum_zip                  object
Cardnum_state                object
zip3                         object
isin 30                      object
avg_risk_amt                 object
Month                        object
Year                         object
abbre_merch_desc             object
isin 100                     object
dtype: object

# 

# Entities

In [147]:
df = data.copy()

In [148]:
df1 = df.copy()
final = df.copy()
df1['check_date'] = df1.Date
df1['check_record'] = df1.Recnum

In [149]:
start = timeit.default_timer()
entities = ['Cardnum', 'Merchnum', 'Merch description', 'Merch zip', 
            'Cardnum_Merchnum', 'Cardnum_zip', 'Cardnum_state', 'zip3', 'avg_risk_amt'] #'Month', 'Year', 'isin 100'
                                                            
for entity in entities:    
    try: print('Run time for the last entity ----------------- {}s'.format(timeit.default_timer() - st))
    except: print('')
    st = timeit.default_timer()    
    ## Day-since variables:   
    df_l = df1[['Recnum', 'Date', entity]]
    df_r = df1[['check_record', 'check_date', entity, 'Amount']]
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)
    temp1 = temp[temp.Recnum > temp.check_record][['Recnum','Date','check_date']]\
                                                   .groupby('Recnum')[['Date', 'check_date']].last()
    mapper = (temp1.Date - temp1.check_date).dt.days
    data[entity + '_day_since'] = data.Recnum.map(mapper)
    data[entity + '_day_since'].fillna(365, inplace = True)
    print('\n' + entity + '_day_since ---> Done')
    for time in [0,1,3,7,14,30]:       
        temp2 = temp[(temp.check_date >= (temp.Date - dt.timedelta(time))) &\
                    (temp.Recnum >= temp.check_record)][['Recnum', entity, 'Amount']]       
        col_name = entity + '_count_' + str(time)    
        mapper2 = temp2.groupby('Recnum')[entity].count()      
        data[col_name] = data.Recnum.map(mapper2)        
        print(col_name + ' ---> Done')       
        data[entity + '_avg_' + str(time)] = data.Recnum.map(temp2.groupby('Recnum')['Amount'].mean())
        data[entity + '_max_' + str(time)] = data.Recnum.map(temp2.groupby('Recnum')['Amount'].max())
        data[entity + '_med_' + str(time)] = data.Recnum.map(temp2.groupby('Recnum')['Amount'].median())
        data[entity + '_total_' + str(time)] = data.Recnum.map(temp2.groupby('Recnum')['Amount'].sum())
        data[entity + '_actual/avg_' + str(time)] = data['Amount'] / data[entity + '_avg_' + str(time)]
        data[entity + '_actual/max_' + str(time)] = data['Amount'] / data[entity + '_max_' + str(time)]
        data[entity + '_actual/med_' + str(time)] = data['Amount'] / data[entity + '_med_' + str(time)]
        data[entity + '_actual/toal_' + str(time)] = data['Amount'] / data[entity + '_total_' + str(time)]       
        print(entity + ' amount variables over past ' + str(time) + ' ---> Done')


Run time for the last entity ----------------- 3782.6992955840005s

Cardnum_day_since ---> Done
Cardnum_count_0 ---> Done
Cardnum amount variables over past 0 ---> Done
Cardnum_count_1 ---> Done
Cardnum amount variables over past 1 ---> Done
Cardnum_count_3 ---> Done
Cardnum amount variables over past 3 ---> Done
Cardnum_count_7 ---> Done
Cardnum amount variables over past 7 ---> Done
Cardnum_count_14 ---> Done
Cardnum amount variables over past 14 ---> Done
Cardnum_count_30 ---> Done
Cardnum amount variables over past 30 ---> Done
Run time for the last entity ----------------- 4.398128207999434s

Merchnum_day_since ---> Done
Merchnum_count_0 ---> Done
Merchnum amount variables over past 0 ---> Done
Merchnum_count_1 ---> Done
Merchnum amount variables over past 1 ---> Done
Merchnum_count_3 ---> Done
Merchnum amount variables over past 3 ---> Done
Merchnum_count_7 ---> Done
Merchnum amount variables over past 7 ---> Done


/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/3797497930.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[col_name] = data.Recnum.map(mapper2)
/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/3797497930.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[entity + '_avg_' + str(time)] = data.Recnum.map(temp2.groupby('Recnum')['Amount'].mean())
/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/3797497930.py:27: PerformanceWarning: DataFrame is highly fragmented.  This

Merchnum_count_14 ---> Done


/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/3797497930.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[entity + '_med_' + str(time)] = data.Recnum.map(temp2.groupby('Recnum')['Amount'].median())
/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/3797497930.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[entity + '_total_' + str(time)] = data.Recnum.map(temp2.groupby('Recnum')['Amount'].sum())
/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/3797497930.py:30: P

Merchnum amount variables over past 14 ---> Done
Merchnum_count_30 ---> Done
Merchnum amount variables over past 30 ---> Done
Run time for the last entity ----------------- 50.318194292000044s


/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/3797497930.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[entity + '_day_since'] = data.Recnum.map(mapper)



Merch description_day_since ---> Done
Merch description_count_0 ---> Done
Merch description amount variables over past 0 ---> Done
Merch description_count_1 ---> Done
Merch description amount variables over past 1 ---> Done
Merch description_count_3 ---> Done
Merch description amount variables over past 3 ---> Done
Merch description_count_7 ---> Done
Merch description amount variables over past 7 ---> Done
Merch description_count_14 ---> Done
Merch description amount variables over past 14 ---> Done
Merch description_count_30 ---> Done
Merch description amount variables over past 30 ---> Done
Run time for the last entity ----------------- 5.665474583000105s

Merch zip_day_since ---> Done
Merch zip_count_0 ---> Done
Merch zip amount variables over past 0 ---> Done
Merch zip_count_1 ---> Done
Merch zip amount variables over past 1 ---> Done
Merch zip_count_3 ---> Done
Merch zip amount variables over past 3 ---> Done
Merch zip_count_7 ---> Done
Merch zip amount variables over past 7 --->

In [150]:
start = timeit.default_timer()
# velocity change variables
for ent in entities:
    for d in ['0', '1']:
        for dd in ['7', '14', '30']:
            data[ent + '_count_' + d + '_by_' + dd] =\
            data[ent + '_count_' + d]/(data[ent + '_count_' + dd])/float(dd)
            
print('run time: {}s'.format(timeit.default_timer() - start))

run time: 0.050257957999747305s


/var/folders/7g/bglwz2w12hn59p4bnxmgjcs00000gn/T/ipykernel_4485/1375481157.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[ent + '_count_' + d + '_by_' + dd] =\


# 

## One hot

In [151]:
data = data.append(pd.get_dummies(data['isin 100']))
data = data.append(pd.get_dummies(data['isin 30']))
data

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,752,786,840,923,926,943,972,973,981,unk
0,1.0,5.142190e+09,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,5.142184e+09,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,5.142132e+09,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,5.142148e+09,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,5.142190e+09,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96748,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96749,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96750,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96751,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [152]:
data

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,...,752,786,840,923,926,943,972,973,981,unk
0,1.0,5.142190e+09,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,5.142184e+09,2006-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,5.142132e+09,2006-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,5.142148e+09,2006-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,5.142190e+09,2006-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96748,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96749,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96750,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96751,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
data.dtypes

Recnum                      float64
Cardnum                     float64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
                          ...      
943                         float64
972                         float64
973                         float64
981                         float64
unk                         float64
Length: 703, dtype: object

In [154]:
len(data.columns)

703

# 